In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [6]:
top = 50
split_pair_path = "../data/split_dev/split_pair_dev_k_{0}.txt".format(top)
train_data_path = "../data/split_dev/train_set.txt"
dev_data_path = "../data/split_dev/dev_set_v1_4.txt"


train_data = pd.read_csv(train_data_path, sep='\t', header=None)
train_label = np.array(train_data.iloc[:,0]).astype('int')
dev_data = pd.read_csv(dev_data_path, sep='\t', header=None)
dev_label = np.array(dev_data.iloc[:,0]).astype('int')
split_data = pd.read_csv(split_pair_path, sep='\t')

In [7]:
split_yes_no_list = np.array(split_data.iloc[:,2])
print("positive:negative = {0}:{1}".format(sum(split_yes_no_list==1), sum(split_yes_no_list==0)))

positive:negative = 127684:1513666


In [8]:
%%time

total_single_split_num = int(len(split_yes_no_list)/top)
new_train_data = pd.DataFrame()
counter = 0
for i in range(0, len(split_yes_no_list), top):
    single_split = split_yes_no_list[i:(i+top)]
    if 1 in single_split:
        new_train_data = new_train_data.append(split_data.iloc[i:(i+top),:])
        counter += 1
print("recall: {0}/{1}={2:.4f}".format(counter, total_single_split_num, (counter/total_single_split_num)))


recall: 14682/32827=0.4473
CPU times: user 25min 51s, sys: 42.6 s, total: 26min 34s
Wall time: 6min 4s


In [9]:
ratio = 1/ (sum(new_train_data.iloc[:,2]==1)/sum(new_train_data.iloc[:,2]==0))
print("positive:negative = {0}:{1}".format(sum(new_train_data.iloc[:,2]==1), sum(new_train_data.iloc[:,2]==0)))
print("ratio: {0}".format(1/ratio))
print("ratio: {0}".format(ratio))

positive:negative = 127684:606416
ratio: 0.21055513047149152
ratio: 4.749349957708092


In [10]:
split_pair_path = "../data/split_dev/new_split_pair_dev_k_{0}.txt".format(top)

new_train_data.to_csv(split_pair_path, sep='\t', index=False)
#new_train_data = pd.read_csv(split_pair_path, sep='\t')
new_train_data = new_train_data.reset_index(drop=True)

In [11]:
# from probability file, get top k candidates
probability_file_path = "/home/weizhuozhang/Downloads/test_results_4.csv"

probability_file = pd.read_csv(probability_file_path, sep='\t',header=None).iloc[:,1]


In [13]:
%%time
new_top = 50
probability_threshold = 1
new_top_candidate_df = pd.DataFrame()
new_candidate_list = []
for i in range(0, len(new_train_data), top):
    new_train_data_single_split = new_train_data[i:(i+top)]
    new_train_label_single_split = np.array(new_train_data_single_split.candidate_label)
    probability_single_split = probability_file[i:(i+top)]
    sorted_index = np.array(np.argsort(probability_single_split)[::-1][:new_top])
    sorted_probability_split = np.array(probability_single_split)[sorted_index]
    new_train_label_single_split = new_train_label_single_split[sorted_index]
    filted_probability_split = sorted_probability_split[sorted_probability_split > probability_threshold]
    if 0 == len(filted_probability_split):
        candidate_label = new_train_label_single_split[0]
        new_candidate_list.append([candidate_label])
    else:
        candidate_label = new_train_label_single_split[:len(filted_probability_split)]
        new_candidate_list.append(candidate_label)
    new_top_candidate_df =new_top_candidate_df.append(new_train_data_single_split.iloc[sorted_index,:])
    

CPU times: user 26min 20s, sys: 34.9 s, total: 26min 55s
Wall time: 5min 59s


In [14]:
#new_top_candidate_df = new_top_candidate_df.reset_index(drop=True)
#np.mean(new_top_candidate_df.pair_result)

In [15]:
new_top_true_label_list = []
new_top_predict_label_list = []
counter = 0
for i in range(0, len(new_top_candidate_df), new_top):
#     new_top_single_split = new_top_candidate_df[i:(i+new_top)]
#     new_top_candidate_split_label = new_top_single_split.candidate_label
#     predict_label = np.argmax(np.bincount(new_top_candidate_split_label))
#     #new_top_candidate_split_label = new_top_single_split.pair_result
#     #new_candidate_label_temp = new_top_candidate_split_label.reset_index(drop=True)
#     #random_index = list(np.random.randint(0, high=new_top, size=1))
#     #predict_label = int(new_candidate_label_temp[random_index])

    new_top_true_label_list.append(new_top_candidate_df.true_label[i])

    predict_label = np.argmax(np.bincount(new_candidate_list[counter]))
    new_top_predict_label_list.append(predict_label)
    counter += 1

In [16]:
new_top_true_label_list = np.array(new_top_true_label_list)
accuracy = accuracy_score(new_top_true_label_list, new_top_predict_label_list)
print("accuracy: {0}".format(accuracy))

accuracy: 0.674908050674295


dev 74367/582173=0.127

split_num = 1
recall: 15795/36746
positive sample: 95259/734920  = 0.1296
negative sample: 639661/734920 = 0.8703

split_num = 2
recall:  /36746

split_num = 3
recall: 36745/36746
positive sample: 94218/734920  = 0.1282
negative sample: 640702/734920 = 0.8717

split_num = 4
recall: 36744/36746
positive sample: 94365/734920  = 0.1284
negative sample: 640555/734920 = 0.8715

split_num = 5
recall: 36744/36746
positive sample: 94157/734920  = 0.1281
negative sample: 640763/734920 = 0.8718

split_num = 6
recall: 36744/36746
positive sample: 93945/734920  = 0.1278
negative sample: 640975/734920 = 0.8721

split_num = 7
recall: 36747/36750
positive sample: 92982/735000  = 0.1265
negative sample: 642018/735000 = 0.8734